In [65]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import time
import random

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.svm import SVC

NUM_OF_ROWS = 5000
ATTACK_NAME = 'PortScan'

In [66]:
pd.set_option('display.max_columns', None)  # Show all columns

---

In [67]:
# import the attack sample dataset
portSamples = pd.read_csv('portscan_open_port_samples_1.csv')
portSamples

,Number of Ports,Average Packet Length,Packet Length Min,Packet Length Max,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,Fwd Packet Length Mean,Fwd Packet Length Min,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Mean,Bwd Packet Length Min,Bwd Packet Length Std,Fwd Segment Size Avg,Bwd Segment Size Avg,Subflow Fwd Bytes,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Max,IAT Mean,IAT Std
0,4990,57.006307,54,60,2.999593,8.997558,130078,26,26.000000,26,0.000000,24,20.002407,20,0.098088,2.002399,0,61.357547,5003,4986,4986,1.460154,6841.058966,1.012127,0.000146,0.010134
1,5003,57.007603,54,60,2.999457,8.996741,130208,26,26.000000,26,0.000000,24,20.003208,20,0.113228,2.003195,0,33.890682,5008,4988,4988,1.520292,6575.052498,1.009149,0.000152,0.010104
2,4985,56.998799,54,60,3.000000,8.999999,129792,26,26.000000,26,0.000000,20,20.000000,20,0.000000,2.000000,0,178.285714,4992,4996,4996,1.420907,7029.312865,0.961251,0.000142,0.009626
3,4937,57.012343,54,60,2.999573,8.997439,130026,26,26.000000,26,0.000000,24,20.002417,20,0.098305,2.002400,0,31.853503,5001,4964,4964,14.153931,704.044686,10.852338,0.001421,0.110534
4,4995,57.004900,54,60,2.999863,8.999176,130182,26,26.000000,26,0.000000,24,20.000801,20,0.056608,2.000799,0,34.882637,5007,4992,4992,1.527861,6544.443000,1.007949,0.000153,0.010090
5,5020,57.015505,54,60,2.999693,8.998159,130598,26,26.000000,26,0.000000,24,20.001608,20,0.080193,2.001593,0,219.124161,5023,4974,4974,1.943438,5143.976667,1.068200,0.000194,0.011385
6,4962,57.004834,54,60,2.999593,8.997560,129246,26,26.000000,26,0.000000,24,20.002420,20,0.098354,2.002414,0,50.964511,4971,4959,4959,1.452540,6836.299800,1.008287,0.000146,0.010125
7,5003,57.012301,54,60,2.999975,8.999849,130520,26,26.000000,26,0.000000,20,20.000000,20,0.000000,2.000000,0,0.000000,5020,4979,4979,0.986577,10135.042330,0.504682,0.000099,0.005080
8,4968,57.007404,54,60,2.999324,8.995943,130156,26,26.000000,26,0.000000,24,20.004010,20,0.126580,2.003995,0,29.487087,5006,4988,4988,6.943519,1439.327787,2.646078,0.000695,0.034648
9,5001,57.003902,54,60,2.999997,8.999985,130104,26,26.000000,26,0.000000,20,20.000000,20,0.000000,2.000000,0,33.155963,5004,4991,4991,1.930330,5177.871039,1.005448,0.000193,0.011300


In [68]:
portSamples.drop(index=17, inplace=True)
portSamples.reset_index(drop=True, inplace=True)
portSamples

,Number of Ports,Average Packet Length,Packet Length Min,Packet Length Max,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,Fwd Packet Length Mean,Fwd Packet Length Min,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Mean,Bwd Packet Length Min,Bwd Packet Length Std,Fwd Segment Size Avg,Bwd Segment Size Avg,Subflow Fwd Bytes,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Max,IAT Mean,IAT Std
0,4990,57.006307,54,60,2.999593,8.997558,130078,26,26.000000,26,0.000000,24,20.002407,20,0.098088,2.002399,0,61.357547,5003,4986,4986,1.460154,6841.058966,1.012127,0.000146,0.010134
1,5003,57.007603,54,60,2.999457,8.996741,130208,26,26.000000,26,0.000000,24,20.003208,20,0.113228,2.003195,0,33.890682,5008,4988,4988,1.520292,6575.052498,1.009149,0.000152,0.010104
2,4985,56.998799,54,60,3.000000,8.999999,129792,26,26.000000,26,0.000000,20,20.000000,20,0.000000,2.000000,0,178.285714,4992,4996,4996,1.420907,7029.312865,0.961251,0.000142,0.009626
3,4937,57.012343,54,60,2.999573,8.997439,130026,26,26.000000,26,0.000000,24,20.002417,20,0.098305,2.002400,0,31.853503,5001,4964,4964,14.153931,704.044686,10.852338,0.001421,0.110534
4,4995,57.004900,54,60,2.999863,8.999176,130182,26,26.000000,26,0.000000,24,20.000801,20,0.056608,2.000799,0,34.882637,5007,4992,4992,1.527861,6544.443000,1.007949,0.000153,0.010090
5,5020,57.015505,54,60,2.999693,8.998159,130598,26,26.000000,26,0.000000,24,20.001608,20,0.080193,2.001593,0,219.124161,5023,4974,4974,1.943438,5143.976667,1.068200,0.000194,0.011385
6,4962,57.004834,54,60,2.999593,8.997560,129246,26,26.000000,26,0.000000,24,20.002420,20,0.098354,2.002414,0,50.964511,4971,4959,4959,1.452540,6836.299800,1.008287,0.000146,0.010125
7,5003,57.012301,54,60,2.999975,8.999849,130520,26,26.000000,26,0.000000,20,20.000000,20,0.000000,2.000000,0,0.000000,5020,4979,4979,0.986577,10135.042330,0.504682,0.000099,0.005080
8,4968,57.007404,54,60,2.999324,8.995943,130156,26,26.000000,26,0.000000,24,20.004010,20,0.126580,2.003995,0,29.487087,5006,4988,4988,6.943519,1439.327787,2.646078,0.000695,0.034648
9,5001,57.003902,54,60,2.999997,8.999985,130104,26,26.000000,26,0.000000,20,20.000000,20,0.000000,2.000000,0,33.155963,5004,4991,4991,1.930330,5177.871039,1.005448,0.000193,0.011300


In [69]:
# find the columns that we need to synthesis data for to produce an attack dataset
columnsToGather = portSamples.replace(0, np.nan) #replace all 0 values with null
columnsToGather = columnsToGather.dropna(how="all", axis=1).columns.tolist() #remove all columns where there are null values
columnsToGather #left with all columns that the values are not 0 (be know for a fact that the data is consistant and there are not missing values in the rows)

['Number of Ports',
 'Average Packet Length',
 'Packet Length Min',
 'Packet Length Max',
 'Packet Length Std',
 'Packet Length Variance',
 'Total Length of Fwd Packet',
 'Fwd Packet Length Max',
 'Fwd Packet Length Mean',
 'Fwd Packet Length Min',
 'Fwd Packet Length Std',
 'Bwd Packet Length Max',
 'Bwd Packet Length Mean',
 'Bwd Packet Length Min',
 'Bwd Packet Length Std',
 'Fwd Segment Size Avg',
 'Subflow Fwd Bytes',
 'SYN Flag Count',
 'ACK Flag Count',
 'RST Flag Count',
 'Flow Duration',
 'Packets Per Second',
 'IAT Max',
 'IAT Mean',
 'IAT Std']

In [70]:
# find an approximate minimum and maximum values of each column and save that data into a dictionary
MinMaxDict = {col: (portSamples[col].min() * 0.85, portSamples[col].max() * 1.075) for col in columnsToGather}
MinMaxDict

{'Number of Ports': (np.float64(2476.9), np.float64(5396.5)),
 'Average Packet Length': (np.float64(48.448978776),
  np.float64(68.9408523695)),
 'Packet Length Min': (np.float64(45.9), np.float64(58.05)),
 'Packet Length Max': (np.float64(51.0), np.float64(79.55)),
 'Packet Length Std': (np.float64(2.54942516115), np.float64(10.7459608509)),
 'Packet Length Variance': (np.float64(7.646551357049999),
  np.float64(107.41923218625)),
 'Total Length of Fwd Packet': (np.float64(102765.0), np.float64(216367.4)),
 'Fwd Packet Length Max': (np.float64(22.099999999999998), np.float64(43.0)),
 'Fwd Packet Length Mean': (np.float64(22.099999999999998),
  np.float64(42.9897476605)),
 'Fwd Packet Length Min': (np.float64(22.099999999999998), np.float64(34.4)),
 'Fwd Packet Length Std': (np.float64(0.0), np.float64(5.192345685749999)),
 'Bwd Packet Length Max': (np.float64(17.0), np.float64(43.0)),
 'Bwd Packet Length Mean': (np.float64(17.0), np.float64(21.514086810749998)),
 'Bwd Packet Length Mi

In [71]:
# change values to int for the columns that should be int (because when calculating approximate values they will be turned into floats)
intColumns = ['Number of Ports', 'Packet Length Min', 'Packet Length Max', 'Total Length of Fwd Packet', 'Fwd Packet Length Max', 'Fwd Packet Length Min', 'Bwd Packet Length Max', 'Bwd Packet Length Min', 'SYN Flag Count', 'ACK Flag Count', 'RST Flag Count']
for key, val in MinMaxDict.items():
    if key in intColumns:
        MinMaxDict[key] = tuple([int(v) for v in val])
MinMaxDict

{'Number of Ports': (2476, 5396),
 'Average Packet Length': (np.float64(48.448978776),
  np.float64(68.9408523695)),
 'Packet Length Min': (45, 58),
 'Packet Length Max': (51, 79),
 'Packet Length Std': (np.float64(2.54942516115), np.float64(10.7459608509)),
 'Packet Length Variance': (np.float64(7.646551357049999),
  np.float64(107.41923218625)),
 'Total Length of Fwd Packet': (102765, 216367),
 'Fwd Packet Length Max': (22, 43),
 'Fwd Packet Length Mean': (np.float64(22.099999999999998),
  np.float64(42.9897476605)),
 'Fwd Packet Length Min': (22, 34),
 'Fwd Packet Length Std': (np.float64(0.0), np.float64(5.192345685749999)),
 'Bwd Packet Length Max': (17, 43),
 'Bwd Packet Length Mean': (np.float64(17.0), np.float64(21.514086810749998)),
 'Bwd Packet Length Min': (17, 21),
 'Bwd Packet Length Std': (np.float64(0.0), np.float64(0.536213333575)),
 'Fwd Segment Size Avg': (np.float64(0.0), np.float64(2.15429484645)),
 'Subflow Fwd Bytes': (np.float64(0.0), np.float64(1547.23214255)),


### Create the dataframe

In [72]:
# creating an empty dataframe before adding values to it
portDataset = pd.DataFrame(np.zeros((NUM_OF_ROWS, len(portSamples.columns))), columns=portSamples.columns)
portDataset.head(3)

,Number of Ports,Average Packet Length,Packet Length Min,Packet Length Max,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,Fwd Packet Length Mean,Fwd Packet Length Min,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Mean,Bwd Packet Length Min,Bwd Packet Length Std,Fwd Segment Size Avg,Bwd Segment Size Avg,Subflow Fwd Bytes,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Max,IAT Mean,IAT Std
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [73]:
# adding zeros to all columns that should not have any values
zeroColumns = [col for col in portSamples.columns if col not in columnsToGather]
for col in zeroColumns:
    portDataset[col] = int(0)
zeroColumns

['Bwd Segment Size Avg']

In [74]:
portDataset.head(3)

,Number of Ports,Average Packet Length,Packet Length Min,Packet Length Max,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,Fwd Packet Length Mean,Fwd Packet Length Min,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Mean,Bwd Packet Length Min,Bwd Packet Length Std,Fwd Segment Size Avg,Bwd Segment Size Avg,Subflow Fwd Bytes,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Max,IAT Mean,IAT Std
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Independant Columns

In [75]:
independant = ['Fwd Packet Length Max', 'Fwd Packet Length Min', 'Fwd Packet Length Mean']

packet_length_max = np.random.randint(MinMaxDict['Fwd Packet Length Max'][0] * 0.9, MinMaxDict['Fwd Packet Length Max'][1] * 1.1, NUM_OF_ROWS)

# Probability of doing X (30%) and Y (70%)
probability = [0.25, 0.75]

# Decide whether to use backward flags (True or False) based on the probability for each row
copy_values = np.random.choice([True, False], size=NUM_OF_ROWS, p=probability)

# Create 'Average Packet Length' and 'Packet Length Min' based on 'Packet Length Max'
packet_length_min = np.where(copy_values, packet_length_max, packet_length_max + np.random.uniform(-4, 4, NUM_OF_ROWS))
packet_length_min = np.minimum(packet_length_min, packet_length_max)

# If True, copy the 'Packet Length Max' values; if False, apply small variation
average_packet_length = np.where(packet_length_max != packet_length_min, (packet_length_max + packet_length_min) / 2, packet_length_min)

# Assign the values to the dataset
portDataset['Fwd Packet Length Max'] = packet_length_max.astype(int)
portDataset['Fwd Packet Length Mean'] = average_packet_length
portDataset['Fwd Packet Length Min'] = packet_length_min.astype(int)

In [76]:
portDataset

,Number of Ports,Average Packet Length,Packet Length Min,Packet Length Max,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,Fwd Packet Length Mean,Fwd Packet Length Min,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Mean,Bwd Packet Length Min,Bwd Packet Length Std,Fwd Segment Size Avg,Bwd Segment Size Avg,Subflow Fwd Bytes,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Max,IAT Mean,IAT Std
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20,19.609972,19,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,22,22.000000,22,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,46,46.000000,46,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,37,36.207180,35,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,24,22.584304,21,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,40,40.000000,40,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,26,25.353392,24,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,43,43.000000,43,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,39,39.000000,39,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [77]:
independent = ['Number of Ports', 'Average Packet Length', 'Packet Length Max', 'Bwd Packet Length Max', 'Subflow Fwd Bytes', 'Bwd Packet Length Mean']

# Generate 'Bwd Packet Length Min' values
bwd_min_low, bwd_min_high = MinMaxDict['Bwd Packet Length Min']
bwd_min_values = np.random.randint(bwd_min_low * 0.9, bwd_min_high * 1.05, size=NUM_OF_ROWS)

for col in independent:
    if col == 'Bwd Packet Length Mean':
        rand_values = np.random.uniform(MinMaxDict[col][0]*0.995, MinMaxDict[col][1] * 1.005, NUM_OF_ROWS)
    else:
        rand_values = np.random.randint(MinMaxDict[col][0] * 0.9, MinMaxDict[col][1] * 1.1, NUM_OF_ROWS)

    portDataset[col] = rand_values

# Ensure that Bwd Packet Length Max is always >= Bwd Packet Length Min
portDataset['Bwd Packet Length Min'] = bwd_min_values
portDataset['Bwd Packet Length Max'] = np.maximum(bwd_min_values, portDataset['Bwd Packet Length Max'])  # Fix inconsistencies

# Ensure that Bwd Packet Length Max is always > Bwd Packet Length Mean > Bwd Packet Length Min
invalid_rows = portDataset['Bwd Packet Length Mean'] > portDataset['Bwd Packet Length Max']

# Compute the correct mean for those rows
corrected_means = (portDataset.loc[invalid_rows, 'Bwd Packet Length Min'] + 
                   portDataset.loc[invalid_rows, 'Bwd Packet Length Max']) / 2

# Update only the invalid rows
portDataset.loc[invalid_rows, 'Bwd Packet Length Mean'] = corrected_means

portDataset

,Number of Ports,Average Packet Length,Packet Length Min,Packet Length Max,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,Fwd Packet Length Mean,Fwd Packet Length Min,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Mean,Bwd Packet Length Min,Bwd Packet Length Std,Fwd Segment Size Avg,Bwd Segment Size Avg,Subflow Fwd Bytes,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Max,IAT Mean,IAT Std
0,4726,49,0.0,77,0.0,0.0,0.0,20,19.609972,19,0.0,37,21.498056,16,0.0,0.0,0,396,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,5050,69,0.0,65,0.0,0.0,0.0,22,22.000000,22,0.0,34,17.362442,19,0.0,0.0,0,1698,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3821,60,0.0,54,0.0,0.0,0.0,46,46.000000,46,0.0,34,20.936116,20,0.0,0.0,0,321,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4234,74,0.0,75,0.0,0.0,0.0,37,36.207180,35,0.0,20,18.913154,18,0.0,0.0,0,1443,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4627,49,0.0,49,0.0,0.0,0.0,24,22.584304,21,0.0,36,18.788851,16,0.0,0.0,0,125,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,3030,60,0.0,82,0.0,0.0,0.0,40,40.000000,40,0.0,19,19.000000,19,0.0,0.0,0,520,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4996,3785,56,0.0,59,0.0,0.0,0.0,26,25.353392,24,0.0,42,17.058847,21,0.0,0.0,0,578,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4997,2682,62,0.0,66,0.0,0.0,0.0,43,43.000000,43,0.0,27,21.311337,18,0.0,0.0,0,551,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4998,5651,71,0.0,74,0.0,0.0,0.0,39,39.000000,39,0.0,29,17.924398,17,0.0,0.0,0,1258,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [78]:
half_and_half = ['Packet Length Std', 'Packet Length Variance', 'Fwd Packet Length Std', 
                 'Flow Duration', 'Total Length of Fwd Packet', 'Bwd Packet Length Std', 'Fwd Segment Size Avg']

for col in half_and_half:
    # Generate random values from the uniform distribution (90% - 110% of min-max range)
    rand_values = np.random.uniform(MinMaxDict[col][0]*0.9, MinMaxDict[col][1]*1.1, NUM_OF_ROWS)
    
    # Generate alternative random values based on column-specific conditions
    if col == 'Packet Length Std':
        usual_values = np.random.uniform(2.9, 3.1, NUM_OF_ROWS)
    elif col == 'Packet Length Variance':
        usual_values = np.random.uniform(8.85, 9.15, NUM_OF_ROWS)
    elif col == 'Fwd Packet Length Std':
        rand_values = np.random.uniform(MinMaxDict[col][0], MinMaxDict[col][1]*1.1, NUM_OF_ROWS)
        usual_values = np.zeros(NUM_OF_ROWS)
    elif col == 'Flow Duration':
        rand_values = np.random.uniform(MinMaxDict[col][0]*0.85, MinMaxDict[col][1], NUM_OF_ROWS)
        usual_values = np.random.uniform(0.85, 8.597, NUM_OF_ROWS)
    elif col == 'Total Length of Fwd Packet':
        usual_values = np.random.randint(MinMaxDict[col][0]*0.9, 150000, NUM_OF_ROWS)
    elif col == 'Bwd Packet Length Std':
        rand_values = np.random.uniform(MinMaxDict[col][0], MinMaxDict[col][1]*1.1, NUM_OF_ROWS)
        usual_values = np.random.uniform(0.035, 0.15, NUM_OF_ROWS)
    elif col == 'Fwd Segment Size Avg':
        rand_values = np.random.uniform(MinMaxDict[col][0]*0.95, MinMaxDict[col][1]*1.05, NUM_OF_ROWS)
        usual_values = np.random.uniform(1.99, 2.01, NUM_OF_ROWS)

    # Choose values randomly (20% from rand_values, 80% from usual_values)
    chosen_values = np.where(np.random.rand(NUM_OF_ROWS) > 0.2, usual_values, rand_values)

    portDataset[col] = chosen_values

portDataset

,Number of Ports,Average Packet Length,Packet Length Min,Packet Length Max,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,Fwd Packet Length Mean,Fwd Packet Length Min,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Mean,Bwd Packet Length Min,Bwd Packet Length Std,Fwd Segment Size Avg,Bwd Segment Size Avg,Subflow Fwd Bytes,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Max,IAT Mean,IAT Std
0,4726,49,0.0,77,2.965797,9.145679,109842.0,20,19.609972,19,0.000000,37,21.498056,16,0.084749,1.993538,0,396,0.0,0.0,0.0,2.787231,0.0,0.0,0.0,0.0
1,5050,69,0.0,65,3.023118,8.860269,97730.0,22,22.000000,22,0.000000,34,17.362442,19,0.038576,0.237312,0,1698,0.0,0.0,0.0,2.027224,0.0,0.0,0.0,0.0
2,3821,60,0.0,54,3.038492,78.838364,135871.0,46,46.000000,46,0.000000,34,20.936116,20,0.149335,0.896127,0,321,0.0,0.0,0.0,7.664975,0.0,0.0,0.0,0.0
3,4234,74,0.0,75,6.467646,8.883248,95134.0,37,36.207180,35,0.000000,20,18.913154,18,0.141855,1.991014,0,1443,0.0,0.0,0.0,5.414687,0.0,0.0,0.0,0.0
4,4627,49,0.0,49,2.997131,9.021089,113405.0,24,22.584304,21,5.297301,36,18.788851,16,0.335562,2.131287,0,125,0.0,0.0,0.0,7.069042,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,3030,60,0.0,82,3.005536,30.339487,103919.0,40,40.000000,40,0.000000,19,19.000000,19,0.142089,2.007318,0,520,0.0,0.0,0.0,7.256998,0.0,0.0,0.0,0.0
4996,3785,56,0.0,59,3.027274,8.936168,99872.0,26,25.353392,24,0.000000,42,17.058847,21,0.100435,2.000357,0,578,0.0,0.0,0.0,2.169228,0.0,0.0,0.0,0.0
4997,2682,62,0.0,66,2.540393,71.472960,130437.0,43,43.000000,43,0.000000,27,21.311337,18,0.136161,1.998773,0,551,0.0,0.0,0.0,8.342118,0.0,0.0,0.0,0.0
4998,5651,71,0.0,74,2.986598,9.042837,141998.0,39,39.000000,39,4.575131,29,17.924398,17,0.048442,1.998159,0,1258,0.0,0.0,0.0,6.459585,0.0,0.0,0.0,0.0


### Same Values

In [79]:
# Generate random values for the 'Flow Duration' column
randValues = np.random.randint(MinMaxDict['Packet Length Min'][0]*0.9, MinMaxDict['Packet Length Min'][1]*1.05, size=NUM_OF_ROWS)

# Assign the random values
portDataset['Packet Length Min'] = randValues

portDataset

,Number of Ports,Average Packet Length,Packet Length Min,Packet Length Max,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,Fwd Packet Length Mean,Fwd Packet Length Min,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Mean,Bwd Packet Length Min,Bwd Packet Length Std,Fwd Segment Size Avg,Bwd Segment Size Avg,Subflow Fwd Bytes,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Max,IAT Mean,IAT Std
0,4726,49,41,77,2.965797,9.145679,109842.0,20,19.609972,19,0.000000,37,21.498056,16,0.084749,1.993538,0,396,0.0,0.0,0.0,2.787231,0.0,0.0,0.0,0.0
1,5050,69,44,65,3.023118,8.860269,97730.0,22,22.000000,22,0.000000,34,17.362442,19,0.038576,0.237312,0,1698,0.0,0.0,0.0,2.027224,0.0,0.0,0.0,0.0
2,3821,60,58,54,3.038492,78.838364,135871.0,46,46.000000,46,0.000000,34,20.936116,20,0.149335,0.896127,0,321,0.0,0.0,0.0,7.664975,0.0,0.0,0.0,0.0
3,4234,74,55,75,6.467646,8.883248,95134.0,37,36.207180,35,0.000000,20,18.913154,18,0.141855,1.991014,0,1443,0.0,0.0,0.0,5.414687,0.0,0.0,0.0,0.0
4,4627,49,49,49,2.997131,9.021089,113405.0,24,22.584304,21,5.297301,36,18.788851,16,0.335562,2.131287,0,125,0.0,0.0,0.0,7.069042,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,3030,60,45,82,3.005536,30.339487,103919.0,40,40.000000,40,0.000000,19,19.000000,19,0.142089,2.007318,0,520,0.0,0.0,0.0,7.256998,0.0,0.0,0.0,0.0
4996,3785,56,57,59,3.027274,8.936168,99872.0,26,25.353392,24,0.000000,42,17.058847,21,0.100435,2.000357,0,578,0.0,0.0,0.0,2.169228,0.0,0.0,0.0,0.0
4997,2682,62,56,66,2.540393,71.472960,130437.0,43,43.000000,43,0.000000,27,21.311337,18,0.136161,1.998773,0,551,0.0,0.0,0.0,8.342118,0.0,0.0,0.0,0.0
4998,5651,71,48,74,2.986598,9.042837,141998.0,39,39.000000,39,4.575131,29,17.924398,17,0.048442,1.998159,0,1258,0.0,0.0,0.0,6.459585,0.0,0.0,0.0,0.0


### Correlation

In [80]:
first_correlation = ['SYN Flag Count', 'ACK Flag Count', 'RST Flag Count']

# finding the correlation between the Number of Ports column to the rest of the columns in order to create new data
independent_col = portSamples[first_correlation[0]].values.reshape(-1, 1) #column 'Number of Ports'
dependent_cols = portSamples[first_correlation[1:]].values #the rest of the columns that are not zeros

# find the scaling factors using least squares function
scaling_factors = np.linalg.lstsq(independent_col, dependent_cols, rcond=None)[0]

scaling_factors = [(name,factor) for name, factor in zip(first_correlation[1:], scaling_factors.flatten())]
for val in scaling_factors:
    print(val)

('ACK Flag Count', np.float64(0.9934935261651908))
('RST Flag Count', np.float64(0.9933592536718514))


In [81]:
# adding the rest of the attack feature values to the dataset at random based on the smaple data
portDataset['SYN Flag Count'] = np.random.randint(MinMaxDict['SYN Flag Count'][0]*0.85, MinMaxDict['SYN Flag Count'][1]*1.1, NUM_OF_ROWS)

for index, row in portDataset.iterrows():
    for col, factor in zip(first_correlation[1:], scaling_factors): #iterating over all rows we need to add values to except 'Number of Ports'
        # calculate a random small delta of the factor for adding some randomness
        delta = random.uniform(factor[1] * 0.01, factor[1] * 0.02) # select a dantom (how much is 1% and 2% from factor)

        # apply the randomness to the calculated number
        updatedFactor = factor[1] + (-1) * delta

        # calculate the value we want to add into the dataset in the given row
        portDataset.loc[index, col] = int(row['SYN Flag Count'] * updatedFactor)

In [82]:
second_correlation = ['Flow Duration', 'Packets Per Second', 'IAT Max', 'IAT Mean', 'IAT Std']

# finding the correlation between the Number of Ports column to the rest of the columns in order to create new data
independent_col = portSamples[second_correlation[0]].values.reshape(-1, 1) #column 'Number of Ports'
dependent_cols = portSamples[second_correlation[1:]].values #the rest of the columns that are not zeros

# find the scaling factors using least squares function
scaling_factors = np.linalg.lstsq(independent_col, dependent_cols, rcond=None)[0]

scaling_factors = [(name,factor) for name, factor in zip(second_correlation[1:], scaling_factors.flatten())]
for val in scaling_factors:
    print(val)

('Packets Per Second', np.float64(78.30849469959969))
('IAT Max', np.float64(0.6321809769264504))
('IAT Mean', np.float64(0.00010107180700798969))
('IAT Std', np.float64(0.006820131130425529))


In [83]:
durationToPacketsCorr = [x * y for x, y in zip(portSamples['Flow Duration'].values, portSamples['Packets Per Second'].values)]
durationToPacketsCorr = np.mean(durationToPacketsCorr)
durationToPacketsCorr

np.float64(9940.88888921109)

In [84]:
# adding the rest of the attack feature values to the dataset at random based on the smaple data
for index, row in portDataset.iterrows():
    for col, factor in zip(second_correlation[1:], scaling_factors): #iterating over all rows we need to add values to except 'Number of Ports'
        if col == 'Packets Per Second':
            delta = random.uniform(durationToPacketsCorr * 0.075, durationToPacketsCorr * 0.1) # select a delta
            updatedFactor = durationToPacketsCorr + random.choice([-1, 1]) * delta
            portDataset.loc[index, col] = updatedFactor / row['Flow Duration']
        else:
            # calculate a random small delta of the factor for adding some randomness
            delta = random.uniform(factor[1] * 0.01, factor[1] * 0.02) # select a dantom (how much is 1% and 2% from factor)

            # apply the randomness to the calculated number
            updatedFactor = factor[1] + random.choice([-1, 1]) * delta

            # calculate the value we want to add into the dataset in the given row
            portDataset.loc[index, col] = row['Flow Duration'] * updatedFactor

In [85]:
# adding a label to the dataset
portDataset['Label'] = ATTACK_NAME

In [86]:
portSamples.describe()

,Number of Ports,Average Packet Length,Packet Length Min,Packet Length Max,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,Fwd Packet Length Mean,Fwd Packet Length Min,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Mean,Bwd Packet Length Min,Bwd Packet Length Std,Fwd Segment Size Avg,Bwd Segment Size Avg,Subflow Fwd Bytes,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Max,IAT Mean,IAT Std
count,18.000000,18.000000,18.0,18.000000,18.000000,18.000000,18.000000,18.000000,18.000000,18.000000,18.000000,18.000000,18.000000,18.0,18.000000,18.000000,18.0,18.000000,18.000000,18.000000,18.000000,18.000000,18.000000,18.000000,18.000000,18.000000
mean,4848.222222,57.459618,54.0,61.555556,3.513727,15.084284,134114.666667,27.555556,26.884977,26.333333,0.283675,24.666667,20.002894,20.0,0.113900,1.874946,0.0,152.519720,4986.777778,4954.444444,4953.777778,5.742802,4944.269115,3.658145,0.000579,0.039199
std,491.206505,1.681917,0.0,4.527332,1.702665,21.622031,17063.620180,4.527332,3.302369,1.414214,1.136493,5.861138,0.003808,0.0,0.146877,0.472518,0.0,330.387193,85.082351,83.260910,83.743695,9.974609,2783.707692,6.380214,0.001009,0.068487
min,2914.000000,56.998799,54.0,60.000000,2.999324,8.995943,120900.000000,26.000000,26.000000,26.000000,0.000000,20.000000,20.000000,20.0,0.000000,0.000000,0.0,0.000000,4650.000000,4641.000000,4641.000000,0.986577,235.283160,0.504682,0.000099,0.005080
25%,4963.500000,57.004851,54.0,60.000000,2.999574,8.997443,130084.500000,26.000000,26.000000,26.000000,0.000000,21.000000,20.000200,20.0,0.014152,2.000000,0.0,33.339643,5001.500000,4966.500000,4966.500000,1.463707,2663.739930,1.006845,0.000147,0.010095
50%,4998.000000,57.007502,54.0,60.000000,2.999775,8.998651,130195.000000,26.000000,26.000000,26.000000,0.000000,24.000000,20.002409,20.0,0.098137,2.001196,0.0,35.352346,5006.500000,4982.500000,4982.500000,1.691956,5844.739540,1.010638,0.000172,0.010876
75%,5003.000000,57.012333,54.0,60.000000,2.999995,8.999973,130390.000000,26.000000,26.000000,26.000000,0.000000,24.000000,20.003212,20.0,0.113304,2.002410,0.0,149.053673,5012.000000,4988.000000,4988.000000,4.097363,6808.495571,1.460001,0.000410,0.021448
max,5020.000000,64.131025,54.0,74.000000,9.996243,99.924867,201272.000000,40.000000,39.990463,32.000000,4.830089,40.000000,20.013104,20.0,0.498803,2.003995,0.0,1439.285714,5030.000000,4996.000000,4996.000000,41.979205,10135.042330,25.286814,0.004251,0.276435


In [87]:
portDataset.describe()

,Number of Ports,Average Packet Length,Packet Length Min,Packet Length Max,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,Fwd Packet Length Mean,Fwd Packet Length Min,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Mean,Bwd Packet Length Min,Bwd Packet Length Std,Fwd Segment Size Avg,Bwd Segment Size Avg,Subflow Fwd Bytes,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Max,IAT Mean,IAT Std
count,5000.000000,5000.000000,5000.000000,5000.00000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.0,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,4073.022400,58.332400,49.544800,64.93280,3.789156,19.796296,129797.775140,32.464000,32.079450,31.505200,0.591111,30.815000,19.003138,18.005800,0.132336,1.816744,0.0,854.372400,4653.889600,4554.305600,4553.309000,8.293165,2600.750310,5.244787,0.000838,0.056598
std,1074.486992,9.215991,5.825563,11.89229,1.992540,26.035715,29371.393029,8.054345,8.095257,8.212372,1.368596,8.901833,1.461555,2.006333,0.114241,0.469848,0.0,491.500439,744.621875,728.821973,728.887468,9.407159,2266.600403,5.957077,0.000951,0.064241
min,2228.000000,43.000000,40.000000,45.00000,2.294999,6.895550,92499.000000,19.000000,17.042983,15.000000,0.000000,15.000000,15.000000,15.000000,0.000931,0.002649,0.0,0.000000,3359.000000,3275.000000,3276.000000,0.745234,198.976968,0.479365,0.000074,0.005172
25%,3142.750000,50.000000,44.000000,55.00000,2.958293,8.937995,109136.873272,26.000000,25.000000,24.000000,0.000000,22.000000,17.820338,16.000000,0.067448,1.991794,0.0,434.750000,4007.750000,3924.000000,3927.000000,3.116311,1281.959222,1.969612,0.000314,0.021337
50%,4049.500000,58.000000,50.000000,65.00000,3.022044,9.038392,125178.500000,32.000000,32.000000,31.000000,0.000000,31.000000,18.966497,18.000000,0.102159,1.998184,0.0,845.500000,4654.500000,4553.500000,4551.500000,5.491449,1824.905658,3.472185,0.000556,0.037355
75%,4980.250000,66.000000,55.000000,75.00000,3.083315,9.130109,141396.210169,39.000000,39.000000,39.000000,0.000000,39.000000,20.245836,20.000000,0.135795,2.004433,0.0,1282.000000,5304.000000,5191.000000,5189.250000,7.763511,3235.874912,4.912814,0.000785,0.052990
max,5934.000000,74.000000,59.000000,85.00000,11.810446,118.037525,237405.151954,46.000000,46.000000,46.000000,5.702594,46.000000,21.619396,21.000000,0.589589,2.260011,0.0,1700.000000,5946.000000,5844.000000,5843.000000,45.117122,14614.229846,29.031852,0.004644,0.313445


In [88]:
intColumns = ['Number of Ports', 'Packet Length Min', 'Packet Length Max', 'Total Length of Fwd Packet', 'Fwd Packet Length Max', 'Fwd Packet Length Min', 'Bwd Packet Length Max', 'Bwd Packet Length Min', 'SYN Flag Count', 'ACK Flag Count', 'RST Flag Count']
for col in intColumns:
    portDataset[col] = portDataset[col].astype(int)

portDataset

,Number of Ports,Average Packet Length,Packet Length Min,Packet Length Max,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,Fwd Packet Length Mean,Fwd Packet Length Min,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Mean,Bwd Packet Length Min,Bwd Packet Length Std,Fwd Segment Size Avg,Bwd Segment Size Avg,Subflow Fwd Bytes,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Max,IAT Mean,IAT Std,Label
0,4726,49,41,77,2.965797,9.145679,109842,20,19.609972,19,0.000000,37,21.498056,16,0.084749,1.993538,0,396,5907,5807,5801,2.787231,3273.765236,1.794953,0.000285,0.018743,PortScan
1,5050,69,44,65,3.023118,8.860269,97730,22,22.000000,22,0.000000,34,17.362442,19,0.038576,0.237312,0,1698,3846,3753,3770,2.027224,4490.565778,1.257174,0.000203,0.013552,PortScan
2,3821,60,58,54,3.038492,78.838364,135871,46,46.000000,46,0.000000,34,20.936116,20,0.149335,0.896127,0,321,4713,4627,4612,7.664975,1192.171873,4.750285,0.000763,0.051567,PortScan
3,4234,74,55,75,6.467646,8.883248,95134,37,36.207180,35,0.000000,20,18.913154,18,0.141855,1.991014,0,1443,4253,4180,4141,5.414687,2007.638874,3.362498,0.000556,0.036261,PortScan
4,4627,49,49,49,2.997131,9.021089,113405,24,22.584304,21,5.297301,36,18.788851,16,0.335562,2.131287,0,125,3417,3343,3351,7.069042,1290.361618,4.527212,0.000725,0.047704,PortScan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,3030,60,45,82,3.005536,30.339487,103919,40,40.000000,40,0.000000,19,19.000000,19,0.142089,2.007318,0,520,5299,5167,5207,7.256998,1479.666937,4.528357,0.000719,0.050228,PortScan
4996,3785,56,57,59,3.027274,8.936168,99872,26,25.353392,24,0.000000,42,17.058847,21,0.100435,2.000357,0,578,4074,3986,3992,2.169228,4978.866062,1.392539,0.000224,0.015017,PortScan
4997,2682,62,56,66,2.540393,71.472960,130437,43,43.000000,43,0.000000,27,21.311337,18,0.136161,1.998773,0,551,5149,5014,5044,8.342118,1074.156260,5.331663,0.000854,0.057631,PortScan
4998,5651,71,48,74,2.986598,9.042837,141998,39,39.000000,39,4.575131,29,17.924398,17,0.048442,1.998159,0,1258,5095,4981,4968,6.459585,1655.538242,4.158495,0.000660,0.044695,PortScan


---

In [89]:
# save the dataset
# portDataset.to_csv('port_scan_open_ports_type1_dataset_new.csv', index=False)

---

### Second sample dataset

---

In [90]:
NUM_OF_ROWS = 5000

In [91]:
# import the attack sample dataset
portSamples = pd.read_csv('portscan_open_port_samples_2.csv')
portSamples

,Number of Ports,Average Packet Length,Packet Length Min,Packet Length Max,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,Fwd Packet Length Mean,Fwd Packet Length Min,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Mean,Bwd Packet Length Min,Bwd Packet Length Std,Fwd Segment Size Avg,Bwd Segment Size Avg,Subflow Fwd Bytes,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Max,IAT Mean,IAT Std
0,3198,59.996550,58,60,0.082993,0.006888,165516,26,26.000000,26,0.000000,24,24.0,24,0.0,2.006912,0.0,26.077832,6366,11,11,9.874222,645.823030,1.088071,0.001549,0.016380
1,4163,59.998305,58,60,0.058205,0.003388,214526,26,26.000000,26,0.000000,24,24.0,24,0.0,2.003394,0.0,28.954785,8251,7,7,9.240387,893.685517,0.058860,0.001119,0.005757
2,2728,59.998156,58,60,0.060706,0.003685,140842,26,26.000000,26,0.000000,24,24.0,24,0.0,2.003692,0.0,31.487145,5417,5,5,5.565601,974.198454,0.049144,0.001027,0.004635
3,3607,59.996941,58,60,0.078163,0.006109,186680,26,26.000000,26,0.000000,24,24.0,24,0.0,2.006128,0.0,26.047161,7180,11,11,10.963778,655.887048,1.096967,0.001525,0.015087
4,4047,59.998252,58,60,0.059109,0.003494,208000,26,26.000000,26,0.000000,24,24.0,24,0.0,2.003500,0.0,29.005717,8000,7,7,8.881072,901.580345,0.067206,0.001109,0.004755
5,3473,59.996815,58,60,0.079745,0.006359,179322,26,26.000000,26,0.000000,24,24.0,24,0.0,2.006380,0.0,26.060456,6897,11,11,10.752290,642.467790,1.097064,0.001557,0.015597
6,4015,59.998231,58,60,0.059455,0.003535,205582,26,26.000000,26,0.000000,24,24.0,24,0.0,2.003541,0.0,29.193695,7907,7,7,8.709820,908.629566,0.058229,0.001101,0.004645
7,3484,59.997114,58,60,0.075924,0.005765,179894,26,26.000000,26,0.000000,24,24.0,24,0.0,2.005781,0.0,26.048943,6919,10,10,10.701885,647.456033,1.092027,0.001545,0.016018
8,4038,59.998251,58,60,0.059120,0.003495,207922,26,26.000000,26,0.000000,24,24.0,24,0.0,2.003501,0.0,29.023171,7997,7,7,8.978749,891.438213,0.070668,0.001122,0.006004
9,3827,59.996868,58,60,0.079083,0.006254,198926,26,26.000000,26,0.000000,24,24.0,24,0.0,2.006274,0.0,26.044252,7651,12,12,10.756960,712.375993,1.095676,0.001404,0.014864


In [92]:
smallPortSamples = portSamples.iloc[[11, 12]]

portSamples.drop(index=11, inplace=True)
portSamples.drop(index=12, inplace=True)
portSamples.drop(index=17, inplace=True)
portSamples.reset_index(drop=True, inplace=True)
portSamples

,Number of Ports,Average Packet Length,Packet Length Min,Packet Length Max,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,Fwd Packet Length Mean,Fwd Packet Length Min,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Mean,Bwd Packet Length Min,Bwd Packet Length Std,Fwd Segment Size Avg,Bwd Segment Size Avg,Subflow Fwd Bytes,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Max,IAT Mean,IAT Std
0,3198,59.996550,58,60,0.082993,0.006888,165516,26,26.0,26,0.0,24,24.0,24,0.0,2.006912,0.0,26.077832,6366,11,11,9.874222,645.823030,1.088071,0.001549,0.016380
1,4163,59.998305,58,60,0.058205,0.003388,214526,26,26.0,26,0.0,24,24.0,24,0.0,2.003394,0.0,28.954785,8251,7,7,9.240387,893.685517,0.058860,0.001119,0.005757
2,2728,59.998156,58,60,0.060706,0.003685,140842,26,26.0,26,0.0,24,24.0,24,0.0,2.003692,0.0,31.487145,5417,5,5,5.565601,974.198454,0.049144,0.001027,0.004635
3,3607,59.996941,58,60,0.078163,0.006109,186680,26,26.0,26,0.0,24,24.0,24,0.0,2.006128,0.0,26.047161,7180,11,11,10.963778,655.887048,1.096967,0.001525,0.015087
4,4047,59.998252,58,60,0.059109,0.003494,208000,26,26.0,26,0.0,24,24.0,24,0.0,2.003500,0.0,29.005717,8000,7,7,8.881072,901.580345,0.067206,0.001109,0.004755
5,3473,59.996815,58,60,0.079745,0.006359,179322,26,26.0,26,0.0,24,24.0,24,0.0,2.006380,0.0,26.060456,6897,11,11,10.752290,642.467790,1.097064,0.001557,0.015597
6,4015,59.998231,58,60,0.059455,0.003535,205582,26,26.0,26,0.0,24,24.0,24,0.0,2.003541,0.0,29.193695,7907,7,7,8.709820,908.629566,0.058229,0.001101,0.004645
7,3484,59.997114,58,60,0.075924,0.005765,179894,26,26.0,26,0.0,24,24.0,24,0.0,2.005781,0.0,26.048943,6919,10,10,10.701885,647.456033,1.092027,0.001545,0.016018
8,4038,59.998251,58,60,0.059120,0.003495,207922,26,26.0,26,0.0,24,24.0,24,0.0,2.003501,0.0,29.023171,7997,7,7,8.978749,891.438213,0.070668,0.001122,0.006004
9,3827,59.996868,58,60,0.079083,0.006254,198926,26,26.0,26,0.0,24,24.0,24,0.0,2.006274,0.0,26.044252,7651,12,12,10.756960,712.375993,1.095676,0.001404,0.014864


In [93]:
# find the columns that we need to synthesis data for to produce an attack dataset
columnsToGather = portSamples.replace(0, np.nan) #replace all 0 values with null
columnsToGather = columnsToGather.dropna(how="all", axis=1).columns.tolist() #remove all columns where there are null values
columnsToGather #left with all columns that the values are not 0 (be know for a fact that the data is consistant and there are not missing values in the rows)

['Number of Ports',
 'Average Packet Length',
 'Packet Length Min',
 'Packet Length Max',
 'Packet Length Std',
 'Packet Length Variance',
 'Total Length of Fwd Packet',
 'Fwd Packet Length Max',
 'Fwd Packet Length Mean',
 'Fwd Packet Length Min',
 'Bwd Packet Length Max',
 'Bwd Packet Length Mean',
 'Bwd Packet Length Min',
 'Fwd Segment Size Avg',
 'Subflow Fwd Bytes',
 'SYN Flag Count',
 'ACK Flag Count',
 'RST Flag Count',
 'Flow Duration',
 'Packets Per Second',
 'IAT Max',
 'IAT Mean',
 'IAT Std']

In [94]:
# find an approximate minimum and maximum values of each column and save that data into a dictionary
MinMaxDict = {col: (portSamples[col].min() * 0.85, portSamples[col].max() * 1.075) for col in columnsToGather}
MinMaxDict

{'Number of Ports': (np.float64(2318.7999999999997), np.float64(4798.8)),
 'Average Packet Length': (np.float64(50.997067586639474),
  np.float64(64.49817752482441)),
 'Packet Length Min': (np.float64(49.3), np.float64(62.349999999999994)),
 'Packet Length Max': (np.float64(51.0), np.float64(64.5)),
 'Packet Length Std': (np.float64(0.04947392386561522),
  np.float64(0.08921783960191092)),
 'Packet Length Variance': (np.float64(0.0028796107560714003),
  np.float64(0.007404486421611374)),
 'Total Length of Fwd Packet': (np.float64(119715.7),
  np.float64(247581.09999999998)),
 'Fwd Packet Length Max': (np.float64(22.099999999999998), np.float64(27.95)),
 'Fwd Packet Length Mean': (np.float64(22.099999999999998), np.float64(27.95)),
 'Fwd Packet Length Min': (np.float64(22.099999999999998), np.float64(27.95)),
 'Bwd Packet Length Max': (np.float64(20.4), np.float64(25.799999999999997)),
 'Bwd Packet Length Mean': (np.float64(20.4), np.float64(25.799999999999997)),
 'Bwd Packet Length Min

In [95]:
# change values to int for the columns that should be int (because when calculating approximate values they will be turned into floats)
intColumns = ['Number of Ports', 'Packet Length Min', 'Packet Length Max', 'Total Length of Fwd Packet', 'Fwd Packet Length Max', 'Fwd Packet Length Min', 'Bwd Packet Length Max', 'Bwd Packet Length Min', 'SYN Flag Count', 'ACK Flag Count', 'RST Flag Count']
for key, val in MinMaxDict.items():
    if key in intColumns:
        MinMaxDict[key] = tuple([int(v) for v in val])
MinMaxDict

{'Number of Ports': (2318, 4798),
 'Average Packet Length': (np.float64(50.997067586639474),
  np.float64(64.49817752482441)),
 'Packet Length Min': (49, 62),
 'Packet Length Max': (51, 64),
 'Packet Length Std': (np.float64(0.04947392386561522),
  np.float64(0.08921783960191092)),
 'Packet Length Variance': (np.float64(0.0028796107560714003),
  np.float64(0.007404486421611374)),
 'Total Length of Fwd Packet': (119715, 247581),
 'Fwd Packet Length Max': (22, 27),
 'Fwd Packet Length Mean': (np.float64(22.099999999999998), np.float64(27.95)),
 'Fwd Packet Length Min': (22, 27),
 'Bwd Packet Length Max': (20, 25),
 'Bwd Packet Length Mean': (np.float64(20.4), np.float64(25.799999999999997)),
 'Bwd Packet Length Min': (20, 25),
 'Fwd Segment Size Avg': (np.float64(1.7028844988486243),
  np.float64(2.157430097392397)),
 'Subflow Fwd Bytes': (np.float64(22.137614558785017),
  np.float64(33.84868097473731)),
 'SYN Flag Count': (4604, 9522),
 'ACK Flag Count': (4, 12),
 'RST Flag Count': (4, 

### Creating the dataframe

In [96]:
# creating an empty dataframe before adding values to it
portDataset2 = pd.DataFrame(np.zeros((NUM_OF_ROWS, len(portSamples.columns))), columns=portSamples.columns)
portDataset2.head(3)

,Number of Ports,Average Packet Length,Packet Length Min,Packet Length Max,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,Fwd Packet Length Mean,Fwd Packet Length Min,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Mean,Bwd Packet Length Min,Bwd Packet Length Std,Fwd Segment Size Avg,Bwd Segment Size Avg,Subflow Fwd Bytes,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Max,IAT Mean,IAT Std
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [97]:
# adding zeros to all columns that should not have any values
zeroColumns = [col for col in portSamples.columns if col not in columnsToGather]
for col in zeroColumns:
    portDataset2[col] = int(0)
zeroColumns

['Fwd Packet Length Std', 'Bwd Packet Length Std', 'Bwd Segment Size Avg']

In [98]:
portSamples

,Number of Ports,Average Packet Length,Packet Length Min,Packet Length Max,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,Fwd Packet Length Mean,Fwd Packet Length Min,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Mean,Bwd Packet Length Min,Bwd Packet Length Std,Fwd Segment Size Avg,Bwd Segment Size Avg,Subflow Fwd Bytes,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Max,IAT Mean,IAT Std
0,3198,59.996550,58,60,0.082993,0.006888,165516,26,26.0,26,0.0,24,24.0,24,0.0,2.006912,0.0,26.077832,6366,11,11,9.874222,645.823030,1.088071,0.001549,0.016380
1,4163,59.998305,58,60,0.058205,0.003388,214526,26,26.0,26,0.0,24,24.0,24,0.0,2.003394,0.0,28.954785,8251,7,7,9.240387,893.685517,0.058860,0.001119,0.005757
2,2728,59.998156,58,60,0.060706,0.003685,140842,26,26.0,26,0.0,24,24.0,24,0.0,2.003692,0.0,31.487145,5417,5,5,5.565601,974.198454,0.049144,0.001027,0.004635
3,3607,59.996941,58,60,0.078163,0.006109,186680,26,26.0,26,0.0,24,24.0,24,0.0,2.006128,0.0,26.047161,7180,11,11,10.963778,655.887048,1.096967,0.001525,0.015087
4,4047,59.998252,58,60,0.059109,0.003494,208000,26,26.0,26,0.0,24,24.0,24,0.0,2.003500,0.0,29.005717,8000,7,7,8.881072,901.580345,0.067206,0.001109,0.004755
5,3473,59.996815,58,60,0.079745,0.006359,179322,26,26.0,26,0.0,24,24.0,24,0.0,2.006380,0.0,26.060456,6897,11,11,10.752290,642.467790,1.097064,0.001557,0.015597
6,4015,59.998231,58,60,0.059455,0.003535,205582,26,26.0,26,0.0,24,24.0,24,0.0,2.003541,0.0,29.193695,7907,7,7,8.709820,908.629566,0.058229,0.001101,0.004645
7,3484,59.997114,58,60,0.075924,0.005765,179894,26,26.0,26,0.0,24,24.0,24,0.0,2.005781,0.0,26.048943,6919,10,10,10.701885,647.456033,1.092027,0.001545,0.016018
8,4038,59.998251,58,60,0.059120,0.003495,207922,26,26.0,26,0.0,24,24.0,24,0.0,2.003501,0.0,29.023171,7997,7,7,8.978749,891.438213,0.070668,0.001122,0.006004
9,3827,59.996868,58,60,0.079083,0.006254,198926,26,26.0,26,0.0,24,24.0,24,0.0,2.006274,0.0,26.044252,7651,12,12,10.756960,712.375993,1.095676,0.001404,0.014864


In [99]:
random_values = ['Average Packet Length', 'Packet Length Std', 'Packet Length Variance', 'Fwd Segment Size Avg', 'Subflow Fwd Bytes']

for col in random_values:
    if col == 'Subflow Fwd Bytes':
        val = np.random.uniform(MinMaxDict[col][0]*0.995, MinMaxDict[col][1]*1.005, size=NUM_OF_ROWS)
    else:
        val = np.random.uniform(MinMaxDict[col][0]*0.9, MinMaxDict[col][1]*1.1, size=NUM_OF_ROWS)
    portDataset2[col] = val

In [100]:
same_value1 = ['Packet Length Min', 'Packet Length Max']
val1 = np.random.randint(MinMaxDict[same_value1[0]][0]*0.9, MinMaxDict[same_value1[0]][1]*1.05, size=NUM_OF_ROWS)

same_value2 = ['Fwd Packet Length Max', 'Fwd Packet Length Mean', 'Fwd Packet Length Min']
val2 = np.random.randint(MinMaxDict[same_value2[0]][0]*0.9, MinMaxDict[same_value2[0]][1]*1.05, size=NUM_OF_ROWS)

same_value3 = ['Bwd Packet Length Min', 'Bwd Packet Length Max', 'Bwd Packet Length Mean']
val3 = np.random.randint(MinMaxDict[same_value3[0]][0]*0.9, MinMaxDict[same_value3[0]][1]*1.05, size=NUM_OF_ROWS)

for col in same_value1:
    if col == 'Packet Length Min':
        portDataset2[col] = val1
    else:
        portDataset2[col] = [val + np.random.randint(2, 8) for val in val1]

for col in same_value2:
    portDataset2[col] = val2

for col in same_value3:
    portDataset2[col] = val3

In [101]:
first_correlation = ['Number of Ports', 'Total Length of Fwd Packet', 'SYN Flag Count', 'ACK Flag Count']

# finding the correlation between the SYN Flag Count column to the rest of the columns in order to create new data
independent_col = portSamples[first_correlation[0]].values.reshape(-1, 1) #column 'SYN Flag Count'
dependent_cols = portSamples[first_correlation[1:]].values #the rest of the columns that are not zeros

# find the scaling factors using least squares function
scaling_factors = np.linalg.lstsq(independent_col, dependent_cols, rcond=None)[0]

scaling_factors = [(name,factor) for name, factor in zip(first_correlation[1:], scaling_factors.flatten())]
for val in scaling_factors:
    print(val)

('Total Length of Fwd Packet', np.float64(51.603268694587804))
('SYN Flag Count', np.float64(1.9847411036379927))
('ACK Flag Count', np.float64(0.0022734815333307983))


In [102]:
portDataset2['Number of Ports'] = np.random.randint(MinMaxDict['Number of Ports'][0]*0.85, MinMaxDict['Number of Ports'][1]*1.1, NUM_OF_ROWS)

for index, row in portDataset2.iterrows():
    for col, factor in scaling_factors: #iterating over all rows we need to add values
        # calculate the value we want to add into the dataset in the given row
        delta = random.uniform(factor * 0.1, factor * 0.2) # select a delta
        updatedFactor = factor + random.choice([-1, 1]) * delta
        portDataset2.loc[index, col] = int(row['Number of Ports'] * updatedFactor)
        if col == 'ACK Flag Count':
            portDataset2.loc[index, 'RST Flag Count'] = int(row['Number of Ports'] * updatedFactor) #copy the value to RST column


In [103]:
# Generate random values for the 'Flow Duration' column
randValues = np.random.uniform(MinMaxDict['Flow Duration'][0]*0.9, MinMaxDict['Flow Duration'][1]*1.05, size=NUM_OF_ROWS)

# Assign the random values
portDataset2['Flow Duration'] = randValues

In [104]:
# finding the correlation between the Number of Ports column to the rest of the columns in order to create new data
secondCorrelation = ['Flow Duration', 'Packets Per Second', 'IAT Max', 'IAT Mean', 'IAT Std']
independent_col = portSamples[secondCorrelation[0]].values.reshape(-1, 1) #column 'Number of Ports'
dependent_cols = portSamples[secondCorrelation[1:]].values #the rest of the columns that are not zeros

# find the scaling factors using least squares function
scaling_factors = np.linalg.lstsq(independent_col, dependent_cols, rcond=None)[0]

scaling_factors = [(name,factor) for name, factor in zip(secondCorrelation[1:], scaling_factors.flatten())]
for val in scaling_factors:
    print(val)

durationToPacketsCorr = [x * y for x, y in zip(portSamples['Flow Duration'].values, portSamples['Packets Per Second'].values)]
durationToPacketsCorr = np.mean(durationToPacketsCorr)
durationToPacketsCorr

('Packets Per Second', np.float64(81.07547767708505))
('IAT Max', np.float64(0.05919619316767066))
('IAT Mean', np.float64(0.0001340921963392785))
('IAT Std', np.float64(0.0010486295444490786))


np.float64(7516.125)

In [105]:
#iterating over all rows we need to add values
for index, row in portDataset2.iterrows():
    for col, factor in scaling_factors: 
        # calculate a random small delta of the factor for adding some randomness
        if col == 'Packets Per Second':
            delta = random.uniform(durationToPacketsCorr * 0.075, durationToPacketsCorr * 0.1) # select a delta
            updatedFactor = durationToPacketsCorr + random.choice([-1, 1]) * delta
            portDataset2.loc[index, col] = updatedFactor / row['Flow Duration']
        else:
            if col == 'IAT Std' or col == 'IAT Max':
                delta = random.uniform(factor * 0.5, factor * 0.8)
                updatedFactor = factor + random.choices([-1, 1], weights=[1, 2], k=1)[0] * delta  
            else:
                delta = random.uniform(factor * 0.1, factor * 0.2) # select a delta
                updatedFactor = factor + random.choice([-1, 1]) * delta
            portDataset2.loc[index, col] = row['Flow Duration'] * updatedFactor

In [106]:
portSamples.describe()

,Number of Ports,Average Packet Length,Packet Length Min,Packet Length Max,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,Fwd Packet Length Mean,Fwd Packet Length Min,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Mean,Bwd Packet Length Min,Bwd Packet Length Std,Fwd Segment Size Avg,Bwd Segment Size Avg,Subflow Fwd Bytes,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Max,IAT Mean,IAT Std
count,16.000000,16.000000,16.0,16.0,16.000000,16.000000,16.00000,16.0,16.0,16.0,16.0,16.0,16.0,16.0,16.0,16.000000,16.0,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000
mean,3782.062500,59.997635,58.0,60.0,0.068096,0.004724,195191.75000,26.0,26.0,26.0,0.0,24.0,24.0,24.0,0.0,2.004736,0.0,27.880730,7507.375000,8.750000,8.750000,9.529528,804.528544,0.518116,0.001279,0.009613
std,431.957478,0.000669,0.0,0.0,0.009604,0.001335,22068.30596,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.001342,0.0,1.769354,848.780998,2.081666,2.081666,1.421124,136.161725,0.526051,0.000226,0.005487
min,2728.000000,59.996550,58.0,60.0,0.058205,0.003388,140842.00000,26.0,26.0,26.0,0.0,24.0,24.0,24.0,0.0,2.003394,0.0,26.044252,5417.000000,5.000000,5.000000,5.565601,642.467790,0.049144,0.001027,0.003061
25%,3570.250000,59.996937,58.0,60.0,0.059371,0.003525,184359.50000,26.0,26.0,26.0,0.0,24.0,24.0,24.0,0.0,2.003531,0.0,26.055982,7090.750000,7.000000,7.000000,8.838259,650.235297,0.061723,0.001093,0.004727
50%,3921.000000,59.998011,58.0,60.0,0.063039,0.003974,202254.00000,26.0,26.0,26.0,0.0,24.0,24.0,24.0,0.0,2.003982,0.0,28.891462,7779.000000,8.000000,8.000000,9.340837,892.561865,0.105977,0.001121,0.006650
75%,4052.750000,59.998236,58.0,60.0,0.078206,0.006116,208747.50000,26.0,26.0,26.0,0.0,24.0,24.0,24.0,0.0,2.006135,0.0,29.042916,8028.750000,11.000000,11.000000,10.753457,915.198956,1.095999,0.001538,0.015236
max,4464.000000,59.998305,58.0,60.0,0.082993,0.006888,230308.00000,26.0,26.0,26.0,0.0,24.0,24.0,24.0,0.0,2.006912,0.0,31.487145,8858.000000,12.000000,12.000000,11.127804,974.198454,1.098471,0.001557,0.016380


In [107]:
portDataset2.describe()

,Number of Ports,Average Packet Length,Packet Length Min,Packet Length Max,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,Fwd Packet Length Mean,Fwd Packet Length Min,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Mean,Bwd Packet Length Min,Bwd Packet Length Std,Fwd Segment Size Avg,Bwd Segment Size Avg,Subflow Fwd Bytes,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Max,IAT Mean,IAT Std
count,5000.000000,5000.000000,5000.000000,5000.00000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.0,5000.000000,5000.000000,5000.000000,5000.0,5000.000000,5000.0,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,3616.380800,58.609259,53.952000,58.47680,0.071499,0.005364,186932.918200,23.037200,23.037200,23.037200,0.0,21.510200,21.510200,21.510200,0.0,1.949421,0.0,27.958543,7149.697800,7.744000,7.744000,8.394311,981.571337,0.602497,0.001129,0.010742
std,955.317576,7.241331,6.042963,6.31413,0.015448,0.001613,57357.948096,2.605835,2.605835,2.605835,0.0,2.274847,2.274847,2.274847,0.0,0.240858,0.0,3.448987,2215.614902,2.541054,2.541054,2.388108,322.461516,0.366864,0.000369,0.006432
min,1970.000000,45.900011,44.000000,46.00000,0.044557,0.002592,81728.000000,19.000000,19.000000,19.000000,0.0,18.000000,18.000000,18.000000,0.0,1.532982,0.0,22.028531,3167.000000,3.000000,3.000000,4.263838,539.873070,0.053009,0.000461,0.000921
25%,2788.750000,52.455832,49.000000,53.00000,0.058023,0.003970,140942.250000,21.000000,21.000000,21.000000,0.0,20.000000,20.000000,20.000000,0.0,1.740210,0.0,24.976004,5356.000000,6.000000,6.000000,6.340917,718.923748,0.211153,0.000829,0.003868
50%,3614.500000,58.591281,54.000000,58.00000,0.071479,0.005341,180738.000000,23.000000,23.000000,23.000000,0.0,22.000000,22.000000,22.000000,0.0,1.952203,0.0,27.928681,6969.500000,8.000000,8.000000,8.330999,901.080721,0.615792,0.001101,0.010942
75%,4451.000000,64.899999,59.000000,64.00000,0.085009,0.006757,224721.000000,25.000000,25.000000,25.000000,0.0,23.250000,23.250000,23.250000,0.0,2.155971,0.0,30.927492,8607.000000,9.000000,9.000000,10.455779,1192.556051,0.924757,0.001385,0.016351
max,5276.000000,70.932649,64.000000,71.00000,0.098130,0.008145,325564.000000,27.000000,27.000000,27.000000,0.0,25.000000,25.000000,25.000000,0.0,2.373168,0.0,34.010346,12513.000000,14.000000,14.000000,12.560366,1923.351520,1.327727,0.002002,0.023695


In [108]:
# adding a label to the dataset
portDataset2['Label'] = ATTACK_NAME

In [109]:
intColumns = ['Number of Ports', 'Packet Length Min', 'Packet Length Max', 'Total Length of Fwd Packet', 'Fwd Packet Length Max', 'Fwd Packet Length Min', 'Bwd Packet Length Max', 'Bwd Packet Length Min', 'SYN Flag Count', 'ACK Flag Count', 'RST Flag Count']
for col in intColumns:
    portDataset2[col] = portDataset2[col].astype(int)

portDataset2['Fwd Packet Length Mean'] = portDataset2['Fwd Packet Length Mean'].astype(float)
portDataset2['Bwd Packet Length Mean'] = portDataset2['Bwd Packet Length Mean'].astype(float)

portDataset2

,Number of Ports,Average Packet Length,Packet Length Min,Packet Length Max,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,Fwd Packet Length Mean,Fwd Packet Length Min,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Mean,Bwd Packet Length Min,Bwd Packet Length Std,Fwd Segment Size Avg,Bwd Segment Size Avg,Subflow Fwd Bytes,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Max,IAT Mean,IAT Std,Label
0,2957,63.543938,60,63,0.055628,0.006584,129925,20,20.0,20,0,23,23.0,23,0,2.066101,0,30.805430,5136,5,5,8.077715,1000.844750,0.853300,0.000915,0.013926,PortScan
1,5036,51.523582,57,62,0.054795,0.005989,214690,23,23.0,23,0,18,18.0,18,0,1.976049,0,24.343121,11252,10,10,5.465408,1495.669478,0.523289,0.000651,0.001279,PortScan
2,2524,46.008660,45,52,0.080321,0.006217,153483,19,19.0,19,0,22,22.0,22,0,1.749177,0,24.535362,5820,4,4,11.408868,719.252209,1.067784,0.001746,0.003186,PortScan
3,4157,61.860942,63,67,0.082959,0.005433,188307,24,24.0,24,0,23,23.0,23,0,2.022991,0,27.250408,9680,11,11,10.345799,786.824149,0.181534,0.001242,0.017431,PortScan
4,3114,65.819213,45,48,0.064858,0.005984,177221,23,23.0,23,0,24,24.0,24,0,2.046813,0,29.708911,7263,5,5,4.334834,1871.406582,0.053009,0.000640,0.007615,PortScan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,2379,62.999925,54,57,0.057893,0.006087,137981,27,27.0,27,0,23,23.0,23,0,2.121445,0,28.703709,4097,4,4,6.027409,1340.802620,0.574514,0.000679,0.009843,PortScan
4996,2046,51.398349,60,67,0.078998,0.007406,126034,27,27.0,27,0,23,23.0,23,0,1.795916,0,32.954272,4590,4,4,11.052785,620.249048,0.993693,0.001706,0.003386,PortScan
4997,3178,51.987087,61,66,0.060529,0.008032,142131,26,26.0,26,0,22,22.0,22,0,1.738975,0,27.461124,5120,8,8,10.472954,662.129421,0.208066,0.001192,0.004470,PortScan
4998,2411,51.428680,62,67,0.051663,0.005734,105880,22,22.0,22,0,23,23.0,23,0,1.755730,0,23.876619,4064,4,4,10.334266,672.645996,0.154765,0.001198,0.016935,PortScan


---

### Creating more rows base on small subset of samples that is slightly different

In [110]:
smallPortSamples
NUM_OF_ROWS = 2500

In [111]:
# creating an empty dataframe before adding values to it
portDataset3 = pd.DataFrame(np.zeros((NUM_OF_ROWS, len(smallPortSamples.columns))), columns=smallPortSamples.columns)

# find the columns that we need to synthesis data for to produce an attack dataset
columnsToGather = smallPortSamples.replace(0, np.nan) #replace all 0 values with null
columnsToGather = columnsToGather.dropna(how="all", axis=1).columns.tolist() #remove all columns where there are null values

# find an approximate minimum and maximum values of each column and save that data into a dictionary
MinMaxDict = {col: (smallPortSamples[col].min() * 0.85, smallPortSamples[col].max() * 1.1) for col in columnsToGather}

# adding zeros to all columns that should not have any values
zeroColumns = [col for col in smallPortSamples.columns if col not in columnsToGather]
for col in zeroColumns:
    portDataset3[col] = int(0)
zeroColumns

['Bwd Packet Length Std', 'Fwd Segment Size Avg', 'Bwd Segment Size Avg']

In [112]:
# change values to int for the columns that should be int (because when calculating approximate values they will be turned into floats)
intColumns = ['Number of Ports', 'Packet Length Min', 'Packet Length Max', 'Total Length of Fwd Packet', 'Fwd Packet Length Max', 'Fwd Packet Length Min', 'Bwd Packet Length Max', 'Bwd Packet Length Min', 'SYN Flag Count', 'ACK Flag Count', 'RST Flag Count']
for key, val in MinMaxDict.items():
    if key in intColumns:
        MinMaxDict[key] = tuple([int(v) for v in val])
MinMaxDict

{'Number of Ports': (850, 1100),
 'Average Packet Length': (np.float64(62.86623634558094),
  np.float64(81.35640326975478)),
 'Packet Length Min': (56, 72),
 'Packet Length Max': (62, 81),
 'Packet Length Std': (np.float64(0.4774370048814869),
  np.float64(0.6185460879795043)),
 'Packet Length Variance': (np.float64(0.2681718748590646),
  np.float64(0.3478175117770441)),
 'Total Length of Fwd Packet': (68238, 177012),
 'Fwd Packet Length Max': (34, 44),
 'Fwd Packet Length Mean': (np.float64(33.96615231458437),
  np.float64(43.956295008691335)),
 'Fwd Packet Length Min': (27, 35),
 'Fwd Packet Length Std': (np.float64(0.47802647711329393),
  np.float64(0.6193114828292697)),
 'Bwd Packet Length Max': (34, 44),
 'Bwd Packet Length Mean': (np.float64(34.0), np.float64(44.0)),
 'Bwd Packet Length Min': (34, 44),
 'Subflow Fwd Bytes': (np.float64(34.118732851085056),
  np.float64(44.264661654135345)),
 'SYN Flag Count': (1703, 4418),
 'ACK Flag Count': (12, 33),
 'RST Flag Count': (4, 11),


In [113]:
random_values = ['Average Packet Length', 'Packet Length Std', 'Packet Length Variance', 'Fwd Packet Length Std', 'Subflow Fwd Bytes', 'Number of Ports']

for col in random_values:
    val = np.random.uniform(MinMaxDict[col][0]*0.95, MinMaxDict[col][1]*1.05, size=NUM_OF_ROWS)
    portDataset3[col] = val

In [114]:
smallPortSamples

,Number of Ports,Average Packet Length,Packet Length Min,Packet Length Max,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,Fwd Packet Length Mean,Fwd Packet Length Min,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Mean,Bwd Packet Length Min,Bwd Packet Length Std,Fwd Segment Size Avg,Bwd Segment Size Avg,Subflow Fwd Bytes,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Max,IAT Mean,IAT Std
11,1000,73.960367,66,74,0.561691,0.315496,160920,40,39.960268,32,0.562384,40,40.0,40,0.0,0.0,0.0,40.139686,4017,30,10,12.911425,312.668816,3.273951,0.003199,0.057305
12,1000,73.960278,66,74,0.562315,0.316198,80280,40,39.960179,32,0.563010,40,40.0,40,0.0,0.0,0.0,40.240602,2004,15,5,4.624056,435.548349,1.099811,0.002297,0.026689


In [115]:
same_value1 = ['Packet Length Min', 'Packet Length Max']
val1 = np.random.randint(MinMaxDict[same_value1[0]][0]*0.9, MinMaxDict[same_value1[0]][1]*1.05, size=NUM_OF_ROWS)

same_value2 = ['Bwd Packet Length Min', 'Bwd Packet Length Max', 'Bwd Packet Length Mean']
val2 = np.random.randint(MinMaxDict[same_value2[0]][0]*0.9, MinMaxDict[same_value2[0]][1]*1.05, size=NUM_OF_ROWS)


for col in same_value1:
    if col == 'Packet Length Min':
        portDataset3[col] = val1
    else:
        portDataset3[col] = [val + np.random.randint(2, 14) for val in val1]

for col in same_value2:
    portDataset3[col] = val2

In [116]:
independant = ['Fwd Packet Length Max', 'Fwd Packet Length Min', 'Fwd Packet Length Mean']

packet_length_max = np.random.randint(MinMaxDict['Fwd Packet Length Max'][0] * 0.9, MinMaxDict['Fwd Packet Length Max'][1] * 1.1, NUM_OF_ROWS)

# Create 'Average Packet Length' and 'Packet Length Min' based on 'Packet Length Max'
packet_length_min = packet_length_max - np.random.randint(2, 16, NUM_OF_ROWS)

# If True, copy the 'Packet Length Max' values; if False, apply small variation
average_packet_length = np.where(packet_length_max != packet_length_min, (packet_length_max + packet_length_min) / 2, packet_length_min)

# Assign the values to the dataset
portDataset3['Fwd Packet Length Max'] = packet_length_max.astype(int)
portDataset3['Fwd Packet Length Mean'] = average_packet_length
portDataset3['Fwd Packet Length Min'] = packet_length_min.astype(int)

In [117]:
portDataset3

,Number of Ports,Average Packet Length,Packet Length Min,Packet Length Max,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,Fwd Packet Length Mean,Fwd Packet Length Min,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Mean,Bwd Packet Length Min,Bwd Packet Length Std,Fwd Segment Size Avg,Bwd Segment Size Avg,Subflow Fwd Bytes,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Max,IAT Mean,IAT Std
0,1126.036214,82.187223,73,78,0.463517,0.331073,0.0,33,29.0,25,0.554856,42,42,42,0,0,0,33.926909,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1090.865683,80.959890,71,75,0.527996,0.321147,0.0,40,38.5,37,0.484706,34,34,34,0,0,0,38.214520,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,973.522107,63.846381,68,77,0.491258,0.287968,0.0,38,33.0,28,0.571512,44,44,44,0,0,0,38.185913,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1085.281171,62.589757,60,66,0.472758,0.286797,0.0,34,28.5,23,0.620955,33,33,33,0,0,0,34.553301,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,978.919958,65.106291,57,67,0.463134,0.294760,0.0,33,29.5,26,0.472074,34,34,34,0,0,0,37.581940,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2495,964.391893,64.429333,70,80,0.612670,0.258958,0.0,33,26.0,19,0.504731,33,33,33,0,0,0,35.095579,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2496,962.215567,80.778340,58,63,0.497923,0.260180,0.0,41,35.5,30,0.592044,44,44,44,0,0,0,39.300013,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2497,1045.248053,67.992235,69,80,0.598282,0.349492,0.0,45,37.5,30,0.572922,30,30,30,0,0,0,43.611062,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2498,946.990809,68.224993,66,68,0.635782,0.310326,0.0,34,30.5,27,0.611497,39,39,39,0,0,0,41.618165,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [118]:
first_correlation = ['SYN Flag Count', 'ACK Flag Count', 'RST Flag Count', 'Total Length of Fwd Packet']

# finding the correlation between the Number of Ports column to the rest of the columns in order to create new data
independent_col = smallPortSamples[first_correlation[0]].values.reshape(-1, 1) #column 'Number of Ports'
dependent_cols = smallPortSamples[first_correlation[1:]].values #the rest of the columns that are not zeros

# find the scaling factors using least squares function
scaling_factors = np.linalg.lstsq(independent_col, dependent_cols, rcond=None)[0]

scaling_factors = [(name,factor) for name, factor in zip(first_correlation[1:], scaling_factors.flatten())]
for val in scaling_factors:
    print(val)
    
# adding the rest of the attack feature values to the dataset at random based on the smaple data
portDataset3['SYN Flag Count'] = np.random.randint(MinMaxDict['SYN Flag Count'][0]*0.9, MinMaxDict['SYN Flag Count'][1]*1.05, NUM_OF_ROWS)

for index, row in portDataset3.iterrows():
    for col, factor in zip(first_correlation[1:], scaling_factors): #iterating over all rows we need to add values to except 'Number of Ports'
        # calculate a random small delta of the factor for adding some randomness
        delta = random.uniform(factor[1] * 0.01, factor[1] * 0.02) # select a dantom (how much is 1% and 2% from factor)

        # apply the randomness to the calculated number
        updatedFactor = factor[1] + (-1) * delta

        # calculate the value we want to add into the dataset in the given row
        portDataset3.loc[index, col] = int(row['SYN Flag Count'] * updatedFactor)

('ACK Flag Count', np.float64(0.007471601883754736))
('RST Flag Count', np.float64(0.002490533961251579))
('Total Length of Fwd Packet', np.float64(40.05977281507004))


In [119]:
portDataset3

,Number of Ports,Average Packet Length,Packet Length Min,Packet Length Max,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,Fwd Packet Length Mean,Fwd Packet Length Min,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Mean,Bwd Packet Length Min,Bwd Packet Length Std,Fwd Segment Size Avg,Bwd Segment Size Avg,Subflow Fwd Bytes,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Max,IAT Mean,IAT Std
0,1126.036214,82.187223,73,78,0.463517,0.331073,180297.0,33,29.0,25,0.554856,42,42,42,0,0,0,33.926909,4576,33.0,11.0,0.0,0.0,0.0,0.0,0.0
1,1090.865683,80.959890,71,75,0.527996,0.321147,163343.0,40,38.5,37,0.484706,34,34,34,0,0,0,38.214520,4133,30.0,10.0,0.0,0.0,0.0,0.0,0.0
2,973.522107,63.846381,68,77,0.491258,0.287968,71439.0,38,33.0,28,0.571512,44,44,44,0,0,0,38.185913,1805,13.0,4.0,0.0,0.0,0.0,0.0,0.0
3,1085.281171,62.589757,60,66,0.472758,0.286797,69884.0,34,28.5,23,0.620955,33,33,33,0,0,0,34.553301,1764,12.0,4.0,0.0,0.0,0.0,0.0,0.0
4,978.919958,65.106291,57,67,0.463134,0.294760,156936.0,33,29.5,26,0.472074,34,34,34,0,0,0,37.581940,3986,29.0,9.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2495,964.391893,64.429333,70,80,0.612670,0.258958,82040.0,33,26.0,19,0.504731,33,33,33,0,0,0,35.095579,2074,15.0,5.0,0.0,0.0,0.0,0.0,0.0
2496,962.215567,80.778340,58,63,0.497923,0.260180,128256.0,41,35.5,30,0.592044,44,44,44,0,0,0,39.300013,3243,23.0,7.0,0.0,0.0,0.0,0.0,0.0
2497,1045.248053,67.992235,69,80,0.598282,0.349492,66568.0,45,37.5,30,0.572922,30,30,30,0,0,0,43.611062,1689,12.0,4.0,0.0,0.0,0.0,0.0,0.0
2498,946.990809,68.224993,66,68,0.635782,0.310326,75678.0,34,30.5,27,0.611497,39,39,39,0,0,0,41.618165,1922,14.0,4.0,0.0,0.0,0.0,0.0,0.0


In [120]:
# Generate random values for the 'Flow Duration' column
randValues = np.random.uniform(MinMaxDict['Flow Duration'][0]*0.9, MinMaxDict['Flow Duration'][1]*1.05, size=NUM_OF_ROWS)

# Assign the random values
portDataset3['Flow Duration'] = randValues

# finding the correlation between the Number of Ports column to the rest of the columns in order to create new data
secondCorrelation = ['Flow Duration', 'Packets Per Second', 'IAT Max', 'IAT Mean', 'IAT Std']
independent_col = smallPortSamples[secondCorrelation[0]].values.reshape(-1, 1) #column 'Number of Ports'
dependent_cols = smallPortSamples[secondCorrelation[1:]].values #the rest of the columns that are not zeros

# find the scaling factors using least squares function
scaling_factors = np.linalg.lstsq(independent_col, dependent_cols, rcond=None)[0]

scaling_factors = [(name, factor) for name, factor in zip(secondCorrelation[1:], scaling_factors.flatten())]
for val in scaling_factors:
    print(val)

durationToPacketsCorr = [x * y for x, y in zip(smallPortSamples['Flow Duration'].values, smallPortSamples['Packets Per Second'].values)]
durationToPacketsCorr = np.mean(durationToPacketsCorr)
durationToPacketsCorr

('Packets Per Second', np.float64(32.17131779515319))
('IAT Max', np.float64(0.2517824943859383))
('IAT Mean', np.float64(0.00027607674297135994))
('IAT Std', np.float64(0.004589897159462554))


np.float64(3025.5)

In [121]:
# adding the rest of the attack feature values to the dataset at random based on the smaple data
for index, row in portDataset3.iterrows():
    for col, factor in zip(secondCorrelation[1:], scaling_factors): #iterating over all rows we need to add values to except 'Number of Ports'
        if col == 'Packets Per Second':
            delta = random.uniform(durationToPacketsCorr * 0.1, durationToPacketsCorr * 0.2) # select a delta
            updatedFactor = durationToPacketsCorr + random.choices([-1, 1], weights=[2, 1], k=1)[0] * delta
            portDataset3.loc[index, col] = updatedFactor / row['Flow Duration']
        elif col == 'IAT Mean':
            delta = random.uniform(factor[1] * 0.5, factor[1] * 0.8) # select a dantom (how much is 1% and 2% from factor)
            updatedFactor = factor[1] + delta
            portDataset3.loc[index, col] = row['Flow Duration'] * updatedFactor
        else:
            # calculate a random small delta of the factor for adding some randomness
            delta = random.uniform(factor[1] * 0.15, factor[1] * 0.35) # select a dantom (how much is 1% and 2% from factor)

            # apply the randomness to the calculated number
            updatedFactor = factor[1] + random.choice([-1, 1]) * delta

            # calculate the value we want to add into the dataset in the given row
            portDataset3.loc[index, col] = row['Flow Duration'] * updatedFactor

In [122]:
intColumns = ['Number of Ports', 'Packet Length Min', 'Packet Length Max', 'Total Length of Fwd Packet', 'Fwd Packet Length Max', 'Fwd Packet Length Min', 'Bwd Packet Length Max', 'Bwd Packet Length Min', 'SYN Flag Count', 'ACK Flag Count', 'RST Flag Count']
for col in intColumns:
    portDataset3[col] = portDataset3[col].astype(int)

portDataset3['Bwd Packet Length Mean'] = portDataset3['Bwd Packet Length Mean'].astype(float)

portDataset3

,Number of Ports,Average Packet Length,Packet Length Min,Packet Length Max,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,Fwd Packet Length Mean,Fwd Packet Length Min,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Mean,Bwd Packet Length Min,Bwd Packet Length Std,Fwd Segment Size Avg,Bwd Segment Size Avg,Subflow Fwd Bytes,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Max,IAT Mean,IAT Std
0,1126,82.187223,73,78,0.463517,0.331073,180297,33,29.0,25,0.554856,42,42.0,42,0,0,0,33.926909,4576,33,11,13.728657,254.759854,4.623706,0.006478,0.076871
1,1090,80.959890,71,75,0.527996,0.321147,163343,40,38.5,37,0.484706,34,34.0,34,0,0,0,38.214520,4133,30,10,11.154225,231.159857,1.911497,0.005210,0.062158
2,973,63.846381,68,77,0.491258,0.287968,71439,38,33.0,28,0.571512,44,44.0,44,0,0,0,38.185913,1805,13,4,10.964419,230.488816,2.291345,0.004611,0.033006
3,1085,62.589757,60,66,0.472758,0.286797,69884,34,28.5,23,0.620955,33,33.0,33,0,0,0,34.553301,1764,12,4,9.500460,286.456756,3.175469,0.004527,0.051484
4,978,65.106291,57,67,0.463134,0.294760,156936,33,29.5,26,0.472074,34,34.0,34,0,0,0,37.581940,3986,29,9,6.930537,381.105795,1.237364,0.003012,0.040722
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2495,964,64.429333,70,80,0.612670,0.258958,82040,33,26.0,19,0.504731,33,33.0,33,0,0,0,35.095579,2074,15,5,6.034229,597.976024,1.021351,0.002631,0.035725
2496,962,80.778340,58,63,0.497923,0.260180,128256,41,35.5,30,0.592044,44,44.0,44,0,0,0,39.300013,3243,23,7,6.276629,565.479098,1.845566,0.002636,0.024153
2497,1045,67.992235,69,80,0.598282,0.349492,66568,45,37.5,30,0.572922,30,30.0,30,0,0,0,43.611062,1689,12,4,11.232021,226.308162,3.363588,0.005242,0.033664
2498,946,68.224993,66,68,0.635782,0.310326,75678,34,30.5,27,0.611497,39,39.0,39,0,0,0,41.618165,1922,14,4,5.737833,471.895754,1.781036,0.002616,0.033023


In [123]:
smallPortSamples

,Number of Ports,Average Packet Length,Packet Length Min,Packet Length Max,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,Fwd Packet Length Mean,Fwd Packet Length Min,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Mean,Bwd Packet Length Min,Bwd Packet Length Std,Fwd Segment Size Avg,Bwd Segment Size Avg,Subflow Fwd Bytes,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Max,IAT Mean,IAT Std
11,1000,73.960367,66,74,0.561691,0.315496,160920,40,39.960268,32,0.562384,40,40.0,40,0.0,0.0,0.0,40.139686,4017,30,10,12.911425,312.668816,3.273951,0.003199,0.057305
12,1000,73.960278,66,74,0.562315,0.316198,80280,40,39.960179,32,0.563010,40,40.0,40,0.0,0.0,0.0,40.240602,2004,15,5,4.624056,435.548349,1.099811,0.002297,0.026689


In [124]:
portDataset3.describe()

,Number of Ports,Average Packet Length,Packet Length Min,Packet Length Max,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,Fwd Packet Length Mean,Fwd Packet Length Min,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Mean,Bwd Packet Length Min,Bwd Packet Length Std,Fwd Segment Size Avg,Bwd Segment Size Avg,Subflow Fwd Bytes,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Max,IAT Mean,IAT Std
count,2500.000000,2500.000000,2500.000000,2500.000000,2500.000000,2500.000000,2500.000000,2500.000000,2500.000000,2500.000000,2500.000000,2500.000000,2500.000000,2500.000000,2500.0,2500.0,2500.0,2500.000000,2500.000000,2500.000000,2500.00000,2500.000000,2500.000000,2500.000000,2500.000000,2500.000000
mean,979.319600,72.483763,62.113200,69.616000,0.552266,0.309974,123073.101200,38.644000,34.429600,30.215200,0.553177,37.378000,37.378000,37.378000,0.0,0.0,0.0,39.478452,3118.706000,22.443200,7.15600,9.122635,370.170346,2.284985,0.004156,0.042270
std,100.362619,7.448294,7.227149,7.985064,0.056159,0.031863,35484.115511,5.132855,5.571576,6.623718,0.056459,4.567303,4.567303,4.567303,0.0,0.0,0.0,4.049074,899.039872,6.622712,2.24073,3.313491,168.663954,1.050137,0.001533,0.019338
min,807.000000,59.723062,50.000000,52.000000,0.453639,0.254886,60299.000000,30.000000,22.500000,15.000000,0.454135,30.000000,30.000000,30.000000,0.0,0.0,0.0,32.418472,1532.000000,11.000000,3.00000,3.543951,165.772243,0.585367,0.001525,0.010809
25%,891.000000,65.975007,56.000000,63.000000,0.504930,0.281768,92554.000000,34.000000,30.000000,25.000000,0.504431,33.000000,33.000000,33.000000,0.0,0.0,0.0,36.045640,2344.500000,17.000000,5.00000,6.180703,242.573781,1.446189,0.002813,0.026733
50%,978.000000,72.434917,62.000000,70.000000,0.551951,0.310930,123354.000000,39.000000,34.500000,30.000000,0.553969,37.000000,37.000000,37.000000,0.0,0.0,0.0,39.446517,3125.500000,22.500000,7.00000,9.078505,316.120372,2.136597,0.004071,0.039254
75%,1065.000000,78.887590,69.000000,76.000000,0.600667,0.337255,154350.750000,43.000000,39.000000,35.000000,0.602495,41.000000,41.000000,41.000000,0.0,0.0,0.0,43.012330,3910.250000,28.000000,9.00000,12.035291,466.475757,2.917084,0.005457,0.053733
max,1154.000000,85.316702,74.000000,87.000000,0.649460,0.365199,183683.000000,47.000000,46.000000,45.000000,0.650236,45.000000,45.000000,45.000000,0.0,0.0,0.0,46.473116,4636.000000,34.000000,11.00000,14.911644,1009.493758,5.044403,0.007368,0.091690


In [125]:
# adding a label to the dataset
portDataset3['Label'] = ATTACK_NAME

---

### Merging all three attack types tougether into one dataset

In [126]:
# sample dos attack dataset
mergedPortDataset = pd.concat([portDataset, portDataset2, portDataset3], axis=0)
mergedPortDataset = mergedPortDataset.sample(frac=1, random_state=42).reset_index(drop=True)
print(mergedPortDataset.shape)

(12500, 27)


In [ ]:
# save the dataset
# mergedPortDataset.to_csv('port_scan_open_ports_dataset_new.csv', index=False)